In [1]:
import pandas as pd
import joblib
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [2]:
fake = pd.read_csv('Datasets\Fake.csv')
true = pd.read_csv('Datasets\True.csv')

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Irteja\AppData\Local\Temp\ipykernel_9788\3296890919.py:1: SyntaxWarning: invalid escape sequence '\F'
  fake = pd.read_csv('Datasets\Fake.csv')
C:\Users\Irteja\AppData\Local\Temp\ipykernel_9788\3296890919.py:2: SyntaxWarning: invalid escape sequence '\T'
  true = pd.read_csv('Datasets\True.csv')


In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class'] = 0
true['class'] = 1

In [6]:
df = pd.concat([fake, true]).sample(frac=1, random_state=42)

In [7]:
# Combine title + text into one column
df['content'] = df['title'].fillna('') + " " + df['text'].fillna('') + " " + df['subject'].fillna('')

In [8]:
df.sample(10)

,title,text,subject,date,class,content
21641,OBAMA’S DREAM IS AMERICA’S NIGHTMARE: 121 Ille...,A different set of laws apply to illegals in f...,left-news,"Jun 16, 2015",0,OBAMA’S DREAM IS AMERICA’S NIGHTMARE: 121 Ille...
1048,"Obama discriminated against coal, nuclear: U.S...",CAPE TOWN (Reuters) - The United States discri...,politicsNews,"October 25, 2017",1,"Obama discriminated against coal, nuclear: U.S..."
10494,Supreme Court temporarily blocks Louisiana abo...,"WASHINGTON (Reuters) - The Supreme Court, two ...",politicsNews,"March 4, 2016",1,Supreme Court temporarily blocks Louisiana abo...
3567,BREAKING: Dakota Access Pipeline STOPPED By A...,The Army Corps of Engineers has denied an ease...,News,"December 4, 2016",0,BREAKING: Dakota Access Pipeline STOPPED By A...
19376,BREAKING: DEMOCRAT Makes Shocking Statement Re...,https://www.youtube.com/watch?v=IioEIUmawRo,left-news,"Dec 31, 2016",0,BREAKING: DEMOCRAT Makes Shocking Statement Re...
2291,NY Daily News Runs Most Explosive Trump Cover...,The New York Daily News has never been subtle ...,News,"March 4, 2017",0,NY Daily News Runs Most Explosive Trump Cover...
12644,Israel defence minister says hopes Palestinian...,JERUSALEM (Reuters) - Israeli Defence Minister...,worldnews,"December 10, 2017",1,Israel defence minister says hopes Palestinian...
13775,CORRUPT STATE DEPARTMENT Deletes Key Info From...,This is seriously concerning! James Rosen call...,politics,"Jun 2, 2016",0,CORRUPT STATE DEPARTMENT Deletes Key Info From...
22147,Al Gore: Climate Change is “PRINCIPAL” Cause o...,21st Century Wire says Can you say: out of to...,US_News,"March 29, 2017",0,Al Gore: Climate Change is “PRINCIPAL” Cause o...
10783,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,politics,"May 26, 2017",0,NANCY PELOSI ARROGANTLY DISMISSES Questions on...


In [9]:
df = df.drop(["title","subject","date","text"], axis = 1)

In [10]:
df.head()

,class,content
22216,0,Ben Stein Calls Out 9th Circuit Court: Committ...
4436,1,Trump drops Steve Bannon from National Securit...
1526,1,Puerto Rico expects U.S. to lift Jones Act shi...
1377,0,OOPS: Trump Just Accidentally Confirmed He Le...
8995,1,Donald Trump heads for Scotland to reopen a go...


In [11]:
df.reset_index(inplace=True)

In [12]:
# --- Preprocessing function ---
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [13]:
df['cleaned'] = df['content'].apply(clean_text)

In [14]:
df.sample(10)

,index,class,content,cleaned
20098,1157,1,Senate Republicans appear to have votes to pav...,senate republicans appear to have votes to pav...
40855,13683,1,Pope urges respect for human rights in Myanmar...,pope urges respect for human rights in myanmar...
19931,3774,1,McConnell: Border tax would likely not pass U....,mcconnell border tax would likely not pass us ...
6098,881,1,Democrats sue to get documents on Trump's Wash...,democrats sue to get documents on trumps washi...
41440,1913,1,Harvey unlikely to boost global property reins...,harvey unlikely to boost global property reins...
4654,3980,1,Trump says would meet North Korean leader unde...,trump says would meet north korean leader unde...
3784,5915,1,Iraq parliament panel asks government to 'reci...,iraq parliament panel asks government to recip...
25464,10240,1,White House opposes bill it says harms FTC con...,white house opposes bill it says harms ftc con...
19822,1656,1,Pelosi calls DACA deal a first step toward com...,pelosi calls daca deal a first step toward com...
17853,3978,0,DAMNING New Poll Shows A BIG Chunk Of Federal...,damning new poll shows a big chunk of federal ...


In [15]:
x =df["cleaned"]
y=df["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25, random_state=42)

In [16]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [17]:
# --- Train SVM ---
svm_model = LinearSVC()
svm_model.fit(xv_train, ytrain)

LinearSVC()

In [18]:
y_pred = svm_model.predict(xv_test)
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5863
           1       1.00      1.00      1.00      5362

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



In [21]:
# --- Save model and vectorizer ---
joblib.dump(svm_model, "svm_model.jb")
joblib.dump(vectorizer, "tfidf_vectorizer.jb")
print("✅ Model and vectorizer saved.")

✅ Model and vectorizer saved.


In [22]:
prediction = svm_model.predict(xv_test)
svm_model.score(xv_test,ytest)

0.9966146993318485